## 구글 드라이브 연결

In [1]:
from google import colab
colab.drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 데이터 경로

In [2]:
DATA_PATH = "/content/drive/MyDrive/dacon/natural_language_based/root_dir/data/" # dacon 에서 제공한  sample_submission.csv 경로
SUBMIT_INPUT_PATH = "/content/drive/MyDrive/dacon/natural_language_based/root_dir/submit/input/" # 앙상블 하기위해 예측확률 npy 파일 저장 폴더
SUBMIT_OUTPUT_PATH = "/content/drive/MyDrive/dacon/natural_language_based/root_dir/submit/output/" #  추론 csv 파일 저장 경로

## 라이브러리

In [3]:
import os
import pandas as pd
import numpy as np
import joblib as jl
from glob import glob
from scipy.stats.mstats import gmean
from scipy import stats

## 제출파일 불러오기

In [4]:
sample_submission = pd.read_csv(f'{DATA_PATH}sample_submission.csv')

## 예측 확률 npy 파일 불러오기


- glob 이용해 SUBMIT_INPUT_PATH 경로에 앙상블 하려는 npy 리스트를 확인한다.

In [5]:
ensemble_files = glob(f"{SUBMIT_INPUT_PATH}*.npy") # 앙상블 파일 이름 리스트
ensemble_files

['/content/drive/MyDrive/dacon/natural_language_based/root_dir/submit/input/soft_ordering_1d_cnn_weights_.npy',
 '/content/drive/MyDrive/dacon/natural_language_based/root_dir/submit/input/embedding_1d_cnn_weights_.npy',
 '/content/drive/MyDrive/dacon/natural_language_based/root_dir/submit/input/Submission_V999_KoELECTRA_LAST_LINUX_0.78834_42_P1.csv.npy']

In [6]:
pred_list = []
for file_name in ensemble_files:
    preds = np.load(f'{file_name}')
    for pred in preds:
        pred_list.append(pred)
        print(pred.shape)
print(len(pred_list))

(43576, 46)
(43576, 46)
(43576, 46)
(43576, 46)
(43576, 46)
(43576, 46)
(43576, 46)
(43576, 46)
(43576, 46)
(43576, 46)
(43576, 46)
(43576, 46)
(43576, 46)
(43576, 46)
(43576, 46)
15


In [7]:
pred_list

[array([[0.7375743 , 0.00587808, 0.00646074, ..., 0.00485165, 0.00545173,
         0.00501023],
        [0.7563754 , 0.0053302 , 0.00526017, ..., 0.00547607, 0.00542002,
         0.0057315 ],
        [0.7592658 , 0.00526209, 0.00529451, ..., 0.00538711, 0.00539186,
         0.00583888],
        ...,
        [0.00821231, 0.02155718, 0.44653597, ..., 0.01032708, 0.01459982,
         0.01981444],
        [0.7617563 , 0.00513814, 0.00522456, ..., 0.00550524, 0.00541389,
         0.00588772],
        [0.7451184 , 0.0046098 , 0.0056329 , ..., 0.00555412, 0.00544992,
         0.0063657 ]], dtype=float32),
 array([[0.75951046, 0.00553403, 0.00535744, ..., 0.00545554, 0.00534974,
         0.00508338],
        [0.7573628 , 0.00542429, 0.00551987, ..., 0.00543564, 0.0053658 ,
         0.00524849],
        [0.7598265 , 0.00522146, 0.0052259 , ..., 0.00535414, 0.0053789 ,
         0.00524905],
        ...,
        [0.00648158, 0.00552822, 0.7268985 , ..., 0.00831893, 0.00500811,
         0.00525572

## 앙상블 후  class 값 반환
- p = 0 : 기하평균
- p = 1 : 산술평균
- p != 0 & p != 1  : 멱평균

In [8]:
def get_ensemble_pred( pred_list ,  p = 1):
    if p == 0:
        pred = gmean(pred_list, axis=0)
    else:
        pred_list = [ pred **p for pred in pred_list ]
        pred = (  np.sum(pred_list,axis=0) / len(pred_list)  ) ** (1/p)
    pred = np.argmax(pred,axis=1)
    return pred

## 산술 평균후 1등확률과 2등확률의 차이가 크지 않을 경우 2등확률의 클래스로 변경

In [9]:
def get_second_label(pred,deviation = 0.1 , label_list = [11 ,17 ,20 ,41 ,42]):
    idx_2 = np.argsort(pred,axis=1)[:,-2] # 2등 클래스
    idx_1 = np.argsort(pred,axis=1)[:,-1] # 1등 클래스
    list_ = []
    i = 0
    for i1,i2 in zip(idx_1,idx_2):
        abs_ = np.abs(pred[i,i1] - pred[i,i2]) # 1등 확률과 2등 확률의 차이
        is_bool = False
        if abs_ < deviation and i1 == 0: # deviation 미만 and 1등 클래스가 0일 경우
            is_bool = True
        list_.append(is_bool)
        i += 1
    cond1 = np.array(list_)

    cond2 = np.isin(np.argsort(pred,axis=1)[:,-2],label_list) # 2등확률의 클래스가 label_list에 있을 경우
    cond = cond1 & cond2 # deviation 미만 and 1등 클래스가 0일 경우 and 2등확률의 클래스가 label_list에 있을 경우
    tmp = pred[cond] # 만족하는 조건의 numpy 배열 생성
    print(f"변경 되는 개수 : {tmp.shape[0]}") # 변경되는 갯수 확인
    return ( cond , np.argsort(tmp,axis=1)[:,-2] ) # bool index 배열과, 조건에 맞는 2등 확률 클래스 배열 리턴

## 예측 확률 리스트 전달 및 앙상블

- 기본값 산술평균

In [10]:
ensemble_type = ""
p = 1

- 클래스값 변경
    - 리더보드 최고 점수를 재현하기위한 조건
    - 1등확률클래스 == 0 and (1등확률 - 2등확률) < 0.3 and 2등확률클래스 in [11 ,12, 16,17 ,20 ,25,26 ,32,41 ,42]

In [23]:
# 이셀을 주석처리하면 산술평균으로 앙상블 합니다.
ensemble_type = "class_change" 
deviation = 0.3  # 1등확률과 2등확률의 차이 threshold
label_list = [11 ,12, 16,17 ,20 ,25,26 ,32,41 ,42] # 검증에서...recall 이 상대적으로 낮은 클래스 10개

In [12]:
if ensemble_type == "class_change":
    pred_test = np.mean(pred_list,axis=0) # 산술평균 
    pred = np.argmax( pred_test ,axis=1) # argmax
    cond , labels = get_second_label( pred_test,deviation = deviation , label_list = label_list ) 
    pred[cond] = labels # 조건에 맞는 클래스만 2등확률의 클래스로 변경
else:
    pred = get_ensemble_pred(pred_list, p=p)
    
pred , pred.shape

변경 되는 개수 : 136


(array([0, 0, 0, ..., 2, 0, 0]), (43576,))

## 제출파일

In [13]:
sample_submission["label"] = pred
sample_submission

,index,label
0,174304,0
1,174305,0
2,174306,0
3,174307,0
4,174308,0
...,...,...
43571,217875,0
43572,217876,0
43573,217877,2
43574,217878,0


In [14]:
sample_submission.to_csv(f'{SUBMIT_OUTPUT_PATH}ensemble_inference.csv', index=False)
print("끝")

끝


# 좋은 대회 열어 주셔서 감사드립니다.